In [18]:
n = 10  # número de guichês
lamb = 0.2  # taxa de entrada de clientes
mi = 10  # taxa de atendimentos a clientes por cada guichê
t = 1000  # intervalo de tempo total sobre o qual se deseja calcular as médias de aceitações e rejeições
th = 10  # tempo que cada guichê demora para atender um cliente
iteracoes = 1  # número de iterações da simulação

In [33]:
from math import exp
from random import random, expovariate
from matplotlib import pyplot as plt

def momentos_chegada(tempo_total, lamb):
    
    for momento in range(0, tempo_total):
        yield random.expovariate(lamb)

In [12]:
class Cliente:
    
    def __init__(self, momento_chegada):
        self.momento_chegada = momento_chegada
    
    def vai_desistir(self):
        if (len(fila) / (len(fila) + n)) > random():
            return True
        return False
    
    def atendido(self, momento):
        self.atendimento = momento

In [ ]:
fila = []
guiches_ocupados = []
atendidos = []
rejeicoes = 0
for chegada in momentos_chegada(t, lamb):
    cliente = Cliente(chegada)
    # Calcule se o cliente irá permanecer na loja
    if cliente.vai_desistir():
        # Se não for, aumente o contador de rejeições e passe para a próxima iteração do loop
        rejeicoes += 1
        continue
    
    # Libere guichês que ficaram vazios desde a última iteração
    for guiche in guiches_ocupados:
        if (guiche.momento_chegada + th) < chegada:
        # Se guichês foram liberados, aumente o contador de clientes atendidos
            guiches_ocupados.remove(guiche)
            guiche.atendido(guiche.momento_chegada + th)
            atendidos.append(guiche)
    
    # Se há um guichê livre, aponte-o para o cliente
    if(len(guiches_ocupados) < n):
        guiches_ocupados.append(cliente)
    else:
        # Se não houver, coloque-o na fila
        fila.append(cliente)

iteracao = {
    'fila': fila,
    'giches_ocupados': guiches_ocupados,
    'atendidos': atendidos,
    'rejeicoes': rejeicoes
}